<a href="https://colab.research.google.com/github/Anirudh071/LivetranslationusingTransformerAcrchitecture/blob/main/Live_Translation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Importing Dataset

In [ ]:
!pip install -q datasets


In [ ]:

# Step 2: Load and clean the dataset
import pandas as pd
from datasets import Dataset

In [ ]:

# Load your uploaded file
df = pd.read_csv('/content/drive/MyDrive/newdata.csv')

In [ ]:
# Clean and rename columns
df.columns = df.columns.str.strip()
df = df.rename(columns={'english_sentence': 'English', 'hindi_sentence': 'Hindi'})

In [ ]:
# Drop rows where English or Hindi is null or empty
df = df.dropna(subset=['English', 'Hindi'])
df = df[(df['English'].str.strip() != '') & (df['Hindi'].str.strip() != '')]


In [ ]:

# Drop unnecessary columns if any
if 'Unnamed: 0' in df.columns:
    df = df.drop(columns=['Unnamed: 0'])

In [ ]:
# Show a preview
print(df.head())

                                             English  \
0  politicians do not have permission to do what ...   
1         I'd like to tell you about one such child,   
2  This percentage is even greater than the perce...   
3  what we really mean is that they're bad at not...   
4  .The ending portion of these Vedas is called U...   

                                               Hindi  
0  राजनीतिज्ञों के पास जो कार्य करना चाहिए, वह कर...  
1  मई आपको ऐसे ही एक बच्चे के बारे में बताना चाहू...  
2   यह प्रतिशत भारत में हिन्दुओं प्रतिशत से अधिक है।  
3     हम ये नहीं कहना चाहते कि वो ध्यान नहीं दे पाते  
4        इन्हीं वेदों का अंतिम भाग उपनिषद कहलाता है।  


In [ ]:
# Step 3: Convert to Hugging Face Dataset
from datasets import Dataset
dataset = Dataset.from_pandas(df[['English', 'Hindi']])

In [ ]:
# Step 4: Load pretrained tokenizer
from transformers import MarianTokenizer

model_name = "Helsinki-NLP/opus-mt-en-hi"
tokenizer = MarianTokenizer.from_pretrained(model_name)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/44.0 [00:00<?, ?B/s]

source.spm:   0%|          | 0.00/812k [00:00<?, ?B/s]

target.spm:   0%|          | 0.00/1.07M [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.10M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.39k [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/transformers/models/marian/tokenization_marian.py:177: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


In [ ]:
def preprocess(example):
    source = example["English"]
    target = example["Hindi"]

    # Tokenize input (English)
    model_inputs = tokenizer(source, padding="max_length", truncation=True, max_length=128)

    # Tokenize target (Hindi)
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(text_target=target, padding="max_length", truncation=True, max_length=128)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs



In [ ]:
# Step 6: Apply tokenization
tokenized_dataset = dataset.map(preprocess)
print(tokenized_dataset[0])


Map:   0%|          | 0/177604 [00:00<?, ? examples/s]

/usr/local/lib/python3.11/dist-packages/transformers/tokenization_utils_base.py:3959: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


{'English': 'politicians do not have permission to do what needs to be done.', 'Hindi': 'राजनीतिज्ञों के पास जो कार्य करना चाहिए, वह करने कि अनुमति नहीं है .', '__index_level_0__': 0, 'input_ids': [21770, 110, 36, 55, 2961, 7, 110, 117, 1874, 7, 42, 846, 3, 0, 61949, 61949, 61949, 61949, 61949, 61949, 61949, 61949, 61949, 61949, 61949, 61949, 61949, 61949, 61949, 61949, 61949, 61949, 61949, 61949, 61949, 61949, 61949, 61949, 61949, 61949, 61949, 61949, 61949, 61949, 61949, 61949, 61949, 61949, 61949, 61949, 61949, 61949, 61949, 61949, 61949, 61949, 61949, 61949, 61949, 61949, 61949, 61949, 61949, 61949, 61949, 61949, 61949, 61949, 61949, 61949, 61949, 61949, 61949, 61949, 61949, 61949, 61949, 61949, 61949, 61949, 61949, 61949, 61949, 61949, 61949, 61949, 61949, 61949, 61949, 61949, 61949, 61949, 61949, 61949, 61949, 61949, 61949, 61949, 61949, 61949, 61949, 61949, 61949, 61949, 61949, 61949, 61949, 61949, 61949, 61949, 61949, 61949, 61949, 61949, 61949, 61949, 61949, 61949, 61949, 6194

In [ ]:
# Split into 90% train, 10% eval
tokenized_dataset = tokenized_dataset.train_test_split(test_size=0.1)

train_dataset = tokenized_dataset["train"]
eval_dataset = tokenized_dataset["test"]


In [ ]:
!pip install -U transformers


In [ ]:
from transformers import Seq2SeqTrainingArguments

training_args = Seq2SeqTrainingArguments(
    output_dir="./results-en-hi",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    weight_decay=0.01,
    save_total_limit=2,
    num_train_epochs=2,
    predict_with_generate=True,
    logging_dir="./logs",
    report_to="none"
)


In [ ]:
# Load the tokenizer and model
from transformers import MarianTokenizer, MarianMTModel

model_name = "Helsinki-NLP/opus-mt-en-hi"
tokenizer = MarianTokenizer.from_pretrained(model_name)
model = MarianMTModel.from_pretrained(model_name)


/usr/local/lib/python3.11/dist-packages/transformers/models/marian/tokenization_marian.py:177: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


pytorch_model.bin:   0%|          | 0.00/306M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/293 [00:00<?, ?B/s]

In [ ]:
from transformers import DataCollatorForSeq2Seq, Seq2SeqTrainer

# Pass the actual model object
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator
)



/tmp/ipython-input-18-4250323646.py:6: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(


In [ ]:
trainer.train()


Step,Training Loss
500,0.915000
1000,0.782100
1500,0.728600
2000,0.725900
2500,0.713700
3000,0.721700
3500,0.696200
4000,0.704100
4500,0.690500
5000,0.668600


/usr/local/lib/python3.11/dist-packages/transformers/modeling_utils.py:3465: UserWarning: Moving the following attributes in the config to the generation config: {'max_length': 512, 'num_beams': 4, 'bad_words_ids': [[61949]]}. You are seeing this warning because you've set generation parameters in the model config, as opposed to in the generation config.
  warnings.warn(


TrainOutput(global_step=39962, training_loss=0.6022877945901634, metrics={'train_runtime': 10326.6702, 'train_samples_per_second': 30.957, 'train_steps_per_second': 3.87, 'total_flos': 1.0836832690372608e+16, 'train_loss': 0.6022877945901634, 'epoch': 2.0})

In [ ]:
# Save the model and tokenizer in the current session
model.save_pretrained("finetuned-en-hi")
tokenizer.save_pretrained("finetuned-en-hi")

# Then copy the folder to your Google Drive
!cp -r finetuned-en-hi /content/drive/MyDrive/


start from here


In [ ]:
from transformers import MarianMTModel, MarianTokenizer

model_path = "/content/drive/MyDrive/finetuned-en-hi"
model = MarianMTModel.from_pretrained(model_path)
tokenizer = MarianTokenizer.from_pretrained(model_path)


/usr/local/lib/python3.11/dist-packages/transformers/models/marian/tokenization_marian.py:175: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


In [ ]:
from transformers import MarianMTModel, MarianTokenizer

# Load the model and tokenizer
model = MarianMTModel.from_pretrained("/content/drive/MyDrive/finetuned-en-hi")
tokenizer = MarianTokenizer.from_pretrained("/content/drive/MyDrive/finetuned-en-hi")

# Translate English text
text = "I am coming home"
inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True)
translated = model.generate(**inputs)
hindi_output = tokenizer.decode(translated[0], skip_special_tokens=True)

print(hindi_output)


मैं घर आ रहा हूँ


In [ ]:
!pip install streamlit pyngrok openai-whisper

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 803.2/803.2 kB 24.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.1/10.1 MB 90.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 86.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 121.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 92.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 57.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.9 MB/s et

In [ ]:
%%writefile app.py
import streamlit as st
import whisper
from transformers import MarianMTModel, MarianTokenizer

# Load Whisper model
st.session_state.whisper_model = whisper.load_model("base")

# Load MarianMT fine-tuned model
model_path = "/content/drive/MyDrive/finetuned-en-hi"
tokenizer = MarianTokenizer.from_pretrained(model_path)
translator_model = MarianMTModel.from_pretrained(model_path)

st.title("🎙️ English Speech/Text to Hindi Translator 🇮🇳")

# Input mode
mode = st.radio("Choose input mode:", ("📝 Text", "🎤 Voice"))

# Process Text Input
if mode == "📝 Text":
    english_text = st.text_area("Enter English text:")
    if st.button("Translate Text"):
        if not english_text.strip():
            st.warning("Please enter some text.")
        else:
            inputs = tokenizer(english_text, return_tensors="pt", truncation=True, padding=True)
            translated = translator_model.generate(**inputs)
            hindi_output = tokenizer.decode(translated[0], skip_special_tokens=True)
            st.success("Hindi Translation:")
            st.write(hindi_output)

# Process Audio Input
else:
    uploaded_file = st.file_uploader("Upload English audio (.wav, .mp3):", type=["wav", "mp3", "m4a"])
    if uploaded_file and st.button("Transcribe & Translate"):
        with open("temp_audio.wav", "wb") as f:
            f.write(uploaded_file.read())

        # Transcribe using Whisper
        result = st.session_state.whisper_model.transcribe("temp_audio.wav")
        english_text = result["text"]

        st.info(f"🗣️ Transcribed Text: {english_text}")

        # Translate using MarianMT
        inputs = tokenizer(english_text, return_tensors="pt", truncation=True, padding=True)
        translated = translator_model.generate(**inputs)
        hindi_output = tokenizer.decode(translated[0], skip_special_tokens=True)

        st.success("🇮🇳 Hindi Translation:")
        st.write(hindi_output)


Writing app.py


In [ ]:
!ngrok config add-authtoken 2yjHlhw0Sic1WS6HZoft9YfyRgm_735r8qqSJhjCa6fHYpjXe

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [ ]:
!pkill streamlit
from pyngrok import ngrok
ngrok.kill()


In [ ]:
# Start streamlit in background
!streamlit run app.py &> /content/log.txt &

# Wait for Streamlit to spin up
import time
time.sleep(5)

# Connect to port 8501 via ngrok
public_url = ngrok.connect(addr="8501", proto="http")
print(f"✅ Your app is live here: {public_url}")


✅ Your app is live here: NgrokTunnel: "https://e0ab-34-125-176-81.ngrok-free.app" -> "http://localhost:8501"
